# 📊 GDP Per Capita Data Exploration & Cleaning

Bu notebook'ta World Bank GDP per capita veri setini keşfedip temizleyeceğiz.

## 🎯 Amaçlar:
- Veri setinin yapısını anlamak
- Missing values analizi
- Temel istatistiksel özellikler
- Veri temizleme işlemleri

In [1]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Display ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Kütüphaneler başarıyla yüklendi!")

✅ Kütüphaneler başarıyla yüklendi!


In [2]:
# Veri setini yükleyelim (relative path kullanıyoruz - GitHub için uygun)
df = pd.read_csv('../data/gdp-per-capita-worldbank.csv')

print(f"📈 Veri seti başarıyla yüklendi!")
print(f"📊 Boyut: {df.shape[0]} satır x {df.shape[1]} sütun")

📈 Veri seti başarıyla yüklendi!
📊 Boyut: 7063 satır x 4 sütun


## 🔍 Veri Seti Yapısı İncelemesi

In [3]:
# İlk 5 satırı görüntüleyelim
print("📋 İlk 5 satır:")
display(df.head())

📋 İlk 5 satır:


,Entity,Code,Year,"GDP per capita, PPP (constant 2021 international $)"
0,Afghanistan,AFG,2000,1617.8264
1,Afghanistan,AFG,2001,1454.1108
2,Afghanistan,AFG,2002,1774.3087
3,Afghanistan,AFG,2003,1815.9282
4,Afghanistan,AFG,2004,1776.9182


In [4]:
# Sütun bilgileri
print("📄 Sütun Bilgileri:")
print(df.info())

📄 Sütun Bilgileri:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7063 entries, 0 to 7062
Data columns (total 4 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               7063 non-null   object 
 1   Code                                                 6605 non-null   object 
 2   Year                                                 7063 non-null   int64  
 3   GDP per capita, PPP (constant 2021 international $)  7063 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 220.8+ KB
None


In [5]:
# Sütun isimlerini kontrol edelim
print("📝 Sütun İsimleri:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

📝 Sütun İsimleri:
1. Entity
2. Code
3. Year
4. GDP per capita, PPP (constant 2021 international $)


In [6]:
# Benzersiz ülke sayısı
print(f"🌍 Toplam ülke/bölge sayısı: {df['Entity'].nunique()}")
print(f"📅 Yıl aralığı: {df['Year'].min()} - {df['Year'].max()}")
print(f"📊 Toplam veri noktası: {len(df)}")

🌍 Toplam ülke/bölge sayısı: 213
📅 Yıl aralığı: 1990 - 2023
📊 Toplam veri noktası: 7063


## 🔎 Missing Values Analizi

In [ ]:
# Missing values kontrolü
print("❌ Missing Values Analizi:")
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})

display(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# GDP sütununun gerçek ismi neyse o sütunu kullanalım
gdp_column = [col for col in df.columns if 'GDP' in col or 'capita' in col or col not in ['Entity', 'Code', 'Year']][0]
print(f"💰 GDP sütunu: {gdp_column}")

# GDP değerlerinin missing olan ülkeler
missing_gdp = df[df[gdp_column].isnull()]
print(f"\n❌ GDP verisi olmayan kayıt sayısı: {len(missing_gdp)}")

if len(missing_gdp) > 0:
    print("\n🔍 GDP verisi eksik olan bazı ülkeler:")
    print(missing_gdp[['Entity', 'Year']].head(10))

## 📊 Temel İstatistiksel Analiz

In [ ]:
# Temel istatistikler
print("📈 GDP Per Capita Temel İstatistikleri:")
display(df[gdp_column].describe())

In [ ]:
# GDP dağılımını görselleştirelim
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df[gdp_column].dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('GDP Per Capita Dağılımı', fontsize=14, fontweight='bold')
axes[0].set_xlabel('GDP Per Capita (USD)')
axes[0].set_ylabel('Frekans')
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(df[gdp_column].dropna(), vert=True)
axes[1].set_title('GDP Per Capita Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('GDP Per Capita (USD)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Outlier analizi
Q1 = df[gdp_column].quantile(0.25)
Q3 = df[gdp_column].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df[gdp_column] < lower_bound) | (df[gdp_column] > upper_bound)]
print(f"\n🔍 Outlier sayısı: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")

## 🌍 Ülkeler ve Yıllar Analizi

In [ ]:
# En çok ve en az veri bulunan ülkeler
country_data_count = df.groupby('Entity').size().sort_values(ascending=False)

print("🏆 En çok veri noktası olan ülkeler:")
print(country_data_count.head(10))

print("\n⚠️ En az veri noktası olan ülkeler:")
print(country_data_count.tail(10))

In [ ]:
# Yıllara göre veri sayısı
yearly_count = df.groupby('Year').size()

plt.figure(figsize=(12, 6))
yearly_count.plot(kind='line', marker='o', linewidth=2, markersize=4)
plt.title('Yıllara Göre Veri Noktası Sayısı', fontsize=14, fontweight='bold')
plt.xlabel('Yıl')
plt.ylabel('Veri Noktası Sayısı')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"📊 En çok veri olan yıl: {yearly_count.idxmax()} ({yearly_count.max()} ülke)")
print(f"📊 En az veri olan yıl: {yearly_count.idxmin()} ({yearly_count.min()} ülke)")

## 🧹 Veri Temizleme

In [ ]:
# Veri temizleme öncesi boyut
print(f"🔢 Temizleme öncesi veri boyutu: {df.shape}")

# 1. Missing GDP değerlerini kaldır
df_clean = df.dropna(subset=[gdp_column]).copy()
print(f"✅ Missing GDP değerleri kaldırıldı: {df_clean.shape}")

# 2. Negatif GDP değerlerini kontrol et
negative_gdp = df_clean[df_clean[gdp_column] < 0]
if len(negative_gdp) > 0:
    print(f"⚠️ Negatif GDP değeri bulunan kayıt: {len(negative_gdp)}")
    df_clean = df_clean[df_clean[gdp_column] >= 0]
    print(f"✅ Negatif değerler kaldırıldı: {df_clean.shape}")
else:
    print("✅ Negatif GDP değeri bulunamadı")

# 3. Aşırı outlier'ları kontrol et (çok uç değerler)
extreme_outliers = df_clean[df_clean[gdp_column] > 200000]  # 200k USD üzeri
if len(extreme_outliers) > 0:
    print(f"\n🔍 Aşırı yüksek GDP değerleri ({len(extreme_outliers)} kayıt):")
    print(extreme_outliers[['Entity', 'Year', gdp_column]].head())

print(f"\n🎉 Final temiz veri boyutu: {df_clean.shape}")
print(f"📉 Kaldırılan kayıt oranı: {((len(df) - len(df_clean)) / len(df) * 100):.1f}%")

In [ ]:
# Temizlenmiş verinin özeti
print("📊 Temizlenmiş Veri Özeti:")
print(f"🌍 Ülke sayısı: {df_clean['Entity'].nunique()}")
print(f"📅 Yıl aralığı: {df_clean['Year'].min()} - {df_clean['Year'].max()}")
print(f"💰 GDP aralığı: ${df_clean[gdp_column].min():,.0f} - ${df_clean[gdp_column].max():,.0f}")
print(f"📈 Ortalama GDP: ${df_clean[gdp_column].mean():,.0f}")
print(f"📊 Medyan GDP: ${df_clean[gdp_column].median():,.0f}")

In [ ]:
# Create cleaned dataset by removing missing GDP values
df_clean = df.dropna(subset=[gdp_column]).copy()

# Remove negative GDP values if any
df_clean = df_clean[df_clean[gdp_column] >= 0]

print(f"📊 Cleaned dataset size: {df_clean.shape}")
print(f"📈 Average GDP: ${df_clean[gdp_column].mean():,.0f}")
print(f"📊 Median GDP: ${df_clean[gdp_column].median():,.0f}")

# Save cleaned dataset
df_clean.to_csv('../data/gdp_cleaned.csv', index=False)
print("💾 Cleaned data saved as '../data/gdp_cleaned.csv'!")

# Set global variable for usage
GDP_COLUMN = gdp_column
print(f"\n🔧 GDP column name: {GDP_COLUMN}")
print("✅ Data exploration and cleaning completed!")

NameError: name 'df_clean' is not defined